# aSPEM Experiment :defining a protocol




In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from  aSPEM import aSPEM
e = aSPEM()

In [2]:
print(e.print_protocol())

TODO


## installing dependencies